In [25]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 수집
- 금리데이터, 뉴스기사, 금통위의사록, 채권분석보고서(05~17년)

### 한은 기준금리 데이터
"""통화정책의 실제 운영
한국은행의 정책금리인 기준금리는 연 8회 금융통화위원회 ‘본회의’에서 결정된다. 기준금리 결정, 즉 통화정책방향 결정을 위한 금융통화위원회의 회의일자는 연간 단위로 미리 정하고 있다. 다만, 경제여건 급변 등으로 정책대응이 필요한 경우에는 임시회의를 개최할 수 있다.
기준금리 결정회의(본회의) 일을 기준으로 할 때, 그 이전 주(週)부터 한국은행 주요 부서 실무진의 각종 비공식 회의를 거친 후, 본회의 전일에는 ‘동향보고회의’가 열린다. 이 동향보고회의에서 한국은행의 주요 부서는 금융통화위원회 위원들에게 국내외 금융·경제 상황에 대한 종합적인 보고를 하며, 이어서 위원들간의 토론이 이루어진다. 본회의는 통상 오전 9시에 열리며 이곳에서 기준금리가 결정되고 통화정책방향 의결문이 작성된다."""(https://www.bok.or.kr/portal/main/contents.do?menuNo=200293)

- 기준금리 추이

https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643

- 기준금리 변동일(통화정책방향 의결문 일자)

'통화정책방향' or'통화정책 방향'으로 색인하여 일자 추리기

https://www.bok.or.kr/portal/bbs/P0000093/list.do?menuNo=200789

https://www.bok.or.kr/portal/bbs/P0000093/list.do?menuNo=200789&searchWrd=%ED%86%B5%ED%99%94%EC%A0%95%EC%B1%85&searchCnd=1&sdate=&edate=&pageIndex=28



In [ ]:
# # 기준금리 추이 가져오기
# import pandas as pd
# table = pd.read_html('https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643', header=0, encoding='utf-8')
# tcopy = table[0]
# tcopy['변경일자']= tcopy['변경일자'].astype(str)
# tcopy['변경일자'] = tcopy['변경일자'] + '년 ' + tcopy['변경일자.1']
# tcopy = tcopy.drop(columns=['변경일자.1'])
# tcopy

,변경일자,기준금리
0,2023년 01월 13일,3.50
1,2022년 11월 24일,3.25
2,2022년 10월 12일,3.00
3,2022년 08월 25일,2.50
4,2022년 07월 13일,2.25
5,2022년 05월 26일,1.75
6,2022년 04월 14일,1.50
7,2022년 01월 14일,1.25
8,2021년 11월 25일,1.00
9,2021년 08월 26일,0.75


In [ ]:
# 의결사항 일자 크롤링

### 금통위의사록


<!-- {
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import requests\n",
    "from bs4 import BeautifulSoup\n",
    "import os\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 저장 위치 설정\n",
    "SAVE_DB_DIR = \"../data/pdf\"\n",
    "if not os.path.exists(SAVE_DB_DIR):\n",
    "    os.makedirs(SAVE_DB_DIR)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "1\n"
     ]
    }
   ],
   "source": [
    "for i in range(1, 2):\n",
    "    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'\n",
    "    params = {\n",
    "    'pageIndex' : i\n",
    "    }\n",
    "    resp = requests.get(b_url, params = params)\n",
    "    soup = BeautifulSoup(resp.content)\n",
    "    \n",
    "    # 하위 수준의 li를 제거하기 위한 코드\n",
    "    # 첫번째 li를 find로 찾은 후 sibling으로 형제수준만 뽑아냄\n",
    "    li_list = []\n",
    "    li= soup.find('div', class_='bdLine type2').find('ul').find('li')\n",
    "    li_li = li.find_next_siblings('li')\n",
    "    li_list.append(li)\n",
    "    li_list.extend(li_li)\n",
    "    \n",
    "    print(i)\n",
    "\n",
    "    # pdf파일 만들기\n",
    "    result_list = []\n",
    "    try:\n",
    "        for x in range(0, len(li_list)):\n",
    "            new_dict = {}\n",
    "            link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')\n",
    "\n",
    "            # 05년 이전 데이터는 pdf파일이 존재하지 않아, 오류\n",
    "            # 05년 부터의 데이터는 pdf와 hwp 파일 모두 존재\n",
    "            # 두 파일의 순서가 제각각이라 if문을 이용하여 pdf파일만 선택함\n",
    "            for link in link_li:\n",
    "\n",
    "                if link.find('a').attrs['title'][-3:] == 'pdf':\n",
    "                    link_u = link.find('a').attrs['href']\n",
    "                    title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text\n",
    "                    url2 = 'http://www.bok.or.kr' + link_u\n",
    "\n",
    "                    # 다운로드 링크를 requests.get을 사용하여 텍스트 출력\n",
    "                    # 추후 pdf가 아닌 txt형식으로 저장하여 다음 단계를 생략할 수 있는지 여부 확인 필\n",
    "                    file_res2 = requests.get(url2)\n",
    "                    file_name = '{}.pdf'.format(title)\n",
    "\n",
    "                    txt_name = os.path.join(SAVE_DB_DIR, file_name)\n",
    "\n",
    "                    with open(txt_name, 'wb') as f:\n",
    "                        f.write(file_res2.content)\n",
    "    except:\n",
    "        print(f\"get url.content error and pass page{i}, {x}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
} -->

### 채권분석보고서(05~17년)

https://finance.naver.com/research/debenture_list.naver?&page=197

-신한

https://www.shinhansec.com/siw/wealth-management/bond-rp/bbs_gibond/view.do

#### import

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib import request

#### pdf 다운로드 크롤러

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib import request
import os

page = 1

comp_list=[]
date_list=[]
fail_list=[]

# 총 페이지 설정
total_page = 191

while True:
    url = 'https://finance.naver.com/research/debenture_list.nhn?' # 주소

    # 파라메터 선언
    params = {
        'page' : page
    }

    # 수프질
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    # 링크덜
    td_link = soup.find('table', class_='type_1').find_all('td', class_='file')
    td_date = soup.find('table', class_='type_1').find_all('td', class_='date')
    tr_comp = soup.find('table', class_= 'type_1').find_all('tr')

    # 증권사 명 추출
    for tr in tr_comp:
        try:
            comp = tr.find_all('td')[1].text
            comp_list.append(comp)
            print(f'{comp}', end=' ')
        except:
            continue

    # 날짜 추출
    for idx, td in enumerate(td_date):
        if idx % 2 == 0:
            date_list.append(td.text)
            print(f'{td.text} 읽음', end=' ')

    # # 파일 다운로드.
    # for idx, td in enumerate(td_link):
    #     fname = f'{page}_{idx+1}.pdf'
    #     save_path = '/content/drive/Shareddrives/4조/data/bondreport/' + fname
    #     try:
    #       file_link = td.find('a')['href']
    #       request.urlretrieve(file_link, save_path)
    #       print(f'{comp_list[idx]} {date_list[idx]} {fname} 다운로드 완료')

    #     except:
    #       fail_list.append(fname)
    #       open(save_path, 'wb').close()  # 빈 PDF 파일 생성
    #       print(f'{comp_list[idx]} {date_list[idx]} {fname} 다운로드 실패')

    if page >= total_page: # 페이지 다 넘기면 탈출
        break

    page += 1 # 페이지 넘기기
    print()

print("다운로드 실패한 파일 목록:", fail_list)


#### pdf파일이 없는 리포트 리스트

- pdf 없는 리포트 요약은 있는데 요약가져와서 대체 할까?

In [ ]:
fail_list

['8_26.pdf',
 '10_22.pdf',
 '13_18.pdf',
 '15_3.pdf',
 '16_18.pdf',
 '18_30.pdf',
 '23_10.pdf',
 '26_3.pdf',
 '27_12.pdf',
 '57_25.pdf']

#### pdf처리를 위한 설치

In [4]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.3 MB/s eta 0:00:00


#### 보고서 발행 회사 목록, 날짜 목록, pdf파일이 없는 리포트 리스트 저장/불러오기

In [ ]:
# import pickle

# with open("/content/drive/Shareddrives/4조/data/bondreport/comp_list.pkl","wb") as f:
#     pickle.dump(comp_list, f)
# with open("/content/drive/Shareddrives/4조/data/bondreport/date_list.pkl","wb") as f:
#     pickle.dump(date_list, f)
# with open("/content/drive/Shareddrives/4조/data/bondreport/fail_list.pkl","wb") as f:
#     pickle.dump(fail_list, f)

In [5]:
import pickle

with open("/content/drive/Shareddrives/4조/data/bondreport/comp_list.pkl","rb") as f:
    comp_list = pickle.load(f)

with open("/content/drive/Shareddrives/4조/data/bondreport/date_list.pkl","rb") as f:
    date_list = pickle.load(f)

with open("/content/drive/Shareddrives/4조/data/bondreport/fail_list.pkl","rb") as f:
    fail_list = pickle.load(f)

#### pdf ==> text 변환 및 전처리

- V1
- V2 : 휴리스틱 접근: 테이블로 추정되는 텍스트 패턴을 찾아서 제거.(연속된 숫자와 구분 기호만 포함된 라인을 찾아서 제거)
- V3 : tabula

In [6]:
# V1
def pdfToText(fname):

    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages, password = password,
                                      caching = caching, check_extractable = True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

In [16]:
# V2
def pdfToText_v2(fname):
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    import re

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                      caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        # 테이블로 추정되는 텍스트 패턴을 찾아서 제거.(연속된 숫자와 구분 기호만 포함된 라인을 찾아서 제거)
    
        lines = text.split('\n')
        cleaned_lines = [line for line in lines if not re.fullmatch(r"[0-9\s,.\-%$]+", line.strip())]
        cleaned_text = '\n'.join(cleaned_lines)

        fp.close()
        device.close()
        retstr.close()

        return cleaned_text
    except Exception as e:
        return str(e)



In [18]:
# pdf에서 테이블 빼는 라이브러리
!pip install tabula-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 45.6 MB/s eta 0:00:00


In [23]:
#  V3 : tabula (위의 라이브러리 무조건 설치 필요)

def pdfToText_v3(fname):
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    import tabula

    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                      caching=caching, check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        # tabula 사용해서 pdf 내 테이블 추출
        tables = tabula.read_pdf(f'/content/drive/Shareddrives/4조/data/bondreport/{fname}', pages='all', multiple_tables=True)

        # 테이블 텍스트로 변환해서 메인텍스트에서 제거
        for table in tables:
            table_text = table.to_string()
            text = text.replace(table_text, '')

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)



#### V1, V2, V3 성능 비교

In [13]:
abc=pdfToText('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [17]:
abc=pdfToText_v2('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [26]:
abc=pdfToText_v3('191_2.pdf')
abc

'Daewoo Bond Brief   \nDaewoo \n\n채권   윤일광(02-768-3058) ikyoon@bestez.com \n\n2008. 12. 23. Tue \n\n채권시황 \n\n(cid:122)  국고채 3년물 금리는 7bp 상승한 3.88%, 5년물 금리는 8bp 상승한 4.24%로 마감 \n\n(cid:122)  연말 북클로징에 들어간 가운데 전반적으로 거래 한산. 얇은 장세 속에 약간의 위험관리 매물이 출회되어도 금리가 밀리는 양상. \n\n(cid:122)  금리 반등의 가장 큰 이유는 당장 다음달부터 크게 늘어나게 되는 국고채 발행물량에 대한 부담. 이번 달에는 3.2조원 발행에 불과했으나 \n\n1월에는 거의 두 배인 6조원 수준으로 예상되며 이에 대한 우려가 일정부분 선반영되는 모습 \n\n(cid:122)  12월 금통위 이후 금리가 단기 급락한 가운데 연말 결산을 맞아 일종의 윈도우 드레싱 양상이 전개되고 있으나, 1월 수급 부담을 앞두고 \n일부 선제적 이익실현 욕구가 대두되면서 매물 출회. 환율이 반등하면서 외인들이 공격적인 국채선물 매도에 나선 것도 약세 조정에 일조 \n\n(cid:122)  한편, 그동안 outperform해 오던 공사채/은행채의 스프레드가 가격 메리트의 소진과 향후 잠재부실(은행채의 경우) 우려 등으로  더 이상 \n\n줄어들지 못하고 오히려 소폭 반등하려는 조짐  \n\n(cid:122)  주요 금리가 키 레벨에서 조정 받기 시작했고 스프레드 역시 추가 축소에 부담을 느끼는 가운데 연말 조심스러운 관리 모드에 진입한 양\n\n상. 큰 폭의 금리 변동 가능성은 높지 않으나, 전반적으로 기간 조정 내지 약간의 약보합(즉 소폭의 금리 반등) 나타날 전망 \n\n관심 차트 및 경제지표 \n\n단기 급락에 따른 피로감에 기간 조정 양상  \n\n  대규모 국채 입찰에 따른 물량 압박으로 금리 상승 \n\n기준금리\n국고채(3년)\n국고채(10년)\n\nCD(91일)\n통안채(1년)\n회사채(무보증3년)AA-

In [27]:
type(abc)

str

#### 전체 데이터 전처리하여 csv파일로 저장

##### 테스트

In [14]:
# import pandas as pd

# text_list=[]
# for idx, comp in enumerate(comp_list[:60]):

#     page = (idx // 30)+1
#     p_no = (idx % 30)+1

#     text=pdfToText(f'{page}_{p_no}.pdf')

#     print(f'{page}_{p_no}.pdf 텍스트 추출...')
#     text=text.replace('\n','')
#     text=text.replace('\r','')
#     text=text.replace('\t','')

#     text_list.append(text)

#     if (idx+1) % 30 == 0:
#         dict_report ={
#             '증권사' : comp_list[idx-29:idx+1],
#             '날짜' : date_list[idx-29:idx+1],
#             '텍스트' : text_list
#         }

#         df = pd.DataFrame(dict_report)
#         df.to_csv(f'bond_report page_{page}.csv', index=False)

#         text_list = []


# rest = len(text_list)

# dict_report ={
#             '증권사' : comp_list[len(comp_list)-rest:],
#             '날짜' : date_list[len(date_list)-rest:],
#             '텍스트' : text_list
#             }

# df = pd.DataFrame(dict_report)
# df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/page_{page}.csv', index=False)

1_1.pdf 텍스트 추출...
1_2.pdf 텍스트 추출...
1_3.pdf 텍스트 추출...
1_4.pdf 텍스트 추출...
1_5.pdf 텍스트 추출...
1_6.pdf 텍스트 추출...
1_7.pdf 텍스트 추출...
1_8.pdf 텍스트 추출...
1_9.pdf 텍스트 추출...
1_10.pdf 텍스트 추출...
1_11.pdf 텍스트 추출...
1_12.pdf 텍스트 추출...
1_13.pdf 텍스트 추출...
1_14.pdf 텍스트 추출...
1_15.pdf 텍스트 추출...
1_16.pdf 텍스트 추출...
1_17.pdf 텍스트 추출...
1_18.pdf 텍스트 추출...
1_19.pdf 텍스트 추출...
1_20.pdf 텍스트 추출...
1_21.pdf 텍스트 추출...
1_22.pdf 텍스트 추출...
1_23.pdf 텍스트 추출...
1_24.pdf 텍스트 추출...
1_25.pdf 텍스트 추출...
1_26.pdf 텍스트 추출...
1_27.pdf 텍스트 추출...
1_28.pdf 텍스트 추출...
1_29.pdf 텍스트 추출...
1_30.pdf 텍스트 추출...
2_1.pdf 텍스트 추출...
2_2.pdf 텍스트 추출...
2_3.pdf 텍스트 추출...
2_4.pdf 텍스트 추출...
2_5.pdf 텍스트 추출...
2_6.pdf 텍스트 추출...
2_7.pdf 텍스트 추출...
2_8.pdf 텍스트 추출...
2_9.pdf 텍스트 추출...
2_10.pdf 텍스트 추출...
2_11.pdf 텍스트 추출...
2_12.pdf 텍스트 추출...
2_13.pdf 텍스트 추출...
2_14.pdf 텍스트 추출...
2_15.pdf 텍스트 추출...
2_16.pdf 텍스트 추출...
2_17.pdf 텍스트 추출...
2_18.pdf 텍스트 추출...
2_19.pdf 텍스트 추출...
2_20.pdf 텍스트 추출...
2_21.pdf 텍스트 추출...
2_22.pdf 텍스트 추출...
2_23.pdf 텍스트 추출...
2_24.pdf 텍스

##### V2를 적용한 전체 파일 csv화

- 빈 pdf를 생성한 부분이 있어 해당 부분 리스트로 체크하여 제거 필요

- 테이블을 깔끔하게 처리하지 못하고 칼럼명이 살아있는 이슈

- Compliance Notice 부분 삭제 방안 필.
(Compliance Notice 뒤의 텍스트는 자른다
본 자료는 /대우/신한
이 자료는 /한화)

In [30]:
import pandas as pd

def save_to_csv():
    
    text_list = []
    for idx, comp in enumerate(comp_list):

        page = (idx // 30) + 1
        p_no = (idx % 30) + 1

        text = pdfToText_v2(f'{page}_{p_no}.pdf')

        print(f'{page}_{p_no}.pdf 텍스트 추출...')
        text = text.replace('\n', '')
        text = text.replace('\r', '')
        text = text.replace('\t', '')

        text_list.append(text)

        if (idx + 1) % 30 == 0 or idx == len(comp_list) - 1:  
            start_idx = idx - 29 if idx + 1 >= 30 else 0  

            dict_report = {
                '증권사': comp_list[start_idx:idx + 1],
                '날짜': date_list[start_idx:idx + 1],
                '텍스트': text_list
            }

            df = pd.DataFrame(dict_report)
            if idx == len(comp_list) - 1:  
                df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/page_{page}.csv', index=False)
            else:
                df.to_csv(f'/content/drive/Shareddrives/4조/data/bondreport/final/bond_report page_{page}.csv', index=False)

            text_list = []

#Run
#save_to_csv()


1_1.pdf 텍스트 추출...
1_2.pdf 텍스트 추출...
1_3.pdf 텍스트 추출...
1_4.pdf 텍스트 추출...
1_5.pdf 텍스트 추출...
1_6.pdf 텍스트 추출...
1_7.pdf 텍스트 추출...
1_8.pdf 텍스트 추출...
1_9.pdf 텍스트 추출...
1_10.pdf 텍스트 추출...
1_11.pdf 텍스트 추출...
1_12.pdf 텍스트 추출...
1_13.pdf 텍스트 추출...
1_14.pdf 텍스트 추출...
1_15.pdf 텍스트 추출...
1_16.pdf 텍스트 추출...
1_17.pdf 텍스트 추출...
1_18.pdf 텍스트 추출...
1_19.pdf 텍스트 추출...
1_20.pdf 텍스트 추출...
1_21.pdf 텍스트 추출...
1_22.pdf 텍스트 추출...
1_23.pdf 텍스트 추출...
1_24.pdf 텍스트 추출...
1_25.pdf 텍스트 추출...
1_26.pdf 텍스트 추출...
1_27.pdf 텍스트 추출...
1_28.pdf 텍스트 추출...
1_29.pdf 텍스트 추출...
1_30.pdf 텍스트 추출...
2_1.pdf 텍스트 추출...
2_2.pdf 텍스트 추출...
2_3.pdf 텍스트 추출...
2_4.pdf 텍스트 추출...
2_5.pdf 텍스트 추출...
2_6.pdf 텍스트 추출...
2_7.pdf 텍스트 추출...
2_8.pdf 텍스트 추출...
2_9.pdf 텍스트 추출...
2_10.pdf 텍스트 추출...
2_11.pdf 텍스트 추출...
2_12.pdf 텍스트 추출...
2_13.pdf 텍스트 추출...
2_14.pdf 텍스트 추출...
2_15.pdf 텍스트 추출...
2_16.pdf 텍스트 추출...
2_17.pdf 텍스트 추출...
2_18.pdf 텍스트 추출...
2_19.pdf 텍스트 추출...
2_20.pdf 텍스트 추출...
2_21.pdf 텍스트 추출...
2_22.pdf 텍스트 추출...
2_23.pdf 텍스트 추출...
2_24.pdf 텍스

OSError: ignored

#### 크롤링은 완료. 전처리는 추가적으로 처리 필요.

### 뉴스기사

- scrapy 사용